In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data : https://www.kaggle.com/datasets/kritikseth/fruit-and-vegetable-image-recognition

In [6]:
import torch;
torch.__version__

'2.0.0'

In [7]:
!pip install imutils
!pip install git+https://github.com/Deci-AI/super-gradients.git@stable
!pip install albumentations 
!pip install split-folders[full]
!pip install --upgrade -q wandb

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25859 sha256=d02b45436c147e712dca61a6c632493c0f16dda01502e125de9b2d1759d925ba
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
  Cloning https://github.com/Deci-AI/super-gradients.git (to revision stable) to /tmp/pip-req-build-5gm17lqx
  Running command git clone --filter=blob:none --quiet https://github.com/Deci-AI/super-gradients.git /tmp/pip-req-build-5gm17lqx
  Running command git checkout -q c1587c542dd67b11ed1c043024adf2c6ed479f22
  Resolved https://github.com/Deci-AI/super-gradients.git to commit c1587c542dd67b11ed1c043024adf2c6ed479f22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 10.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (set

In [11]:
from PIL import Image
import os
import numpy as np


import random
import pathlib
import wandb
import math
import random
from typing import Dict, List,Tuple
import requests

import numpy as np
import matplotlib.pyplot as plt
import glob
from pathlib import Path, PurePath
import pathlib
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms

from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split

from imutils import paths

import splitfolders
import textwrap

import super_gradients
from super_gradients.common.object_names import Models
from super_gradients.training import Trainer
from super_gradients.training import training_hyperparams
from super_gradients.training.metrics.classification_metrics import Accuracy, Top5
from super_gradients.training.utils.early_stopping import EarlyStop
from super_gradients.training import models
from super_gradients.training.utils.callbacks import Phase

import splitfolders
import textwrap
import torch
from torch.utils.data import Dataset
import json
import os
from PIL import Image
from pathlib import Path
import cv2
import matplotlib.pyplot as plt
from glob import glob
# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2
import tensorflow as tf
import plotly.io as pio
print(pio.renderers)
import gc
import requests
import torch
from PIL import Image
# Visualization Imports
from matplotlib.colors import ListedColormap
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import tifffile as tif
import seaborn as sns
from PIL import Image, ImageEnhance; Image.MAX_IMAGE_PIXELS = 5_000_000_000;
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
from matplotlib import animation, rc; rc('animation', html='jshtml')
import plotly
import PIL
import cv2
import shutil

		– MATPLOTLIB VERSION: 3.7.2
Renderers configuration
-----------------------
    Default renderer: 'kaggle'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

		– MATPLOTLIB VERSION: 3.7.2


In [9]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)
wandb_name = wandb.util.generate_id()
wandb.init(project="Fruit classification with regnetY800", name=wandb_name)


		– MATPLOTLIB VERSION: 3.7.2
Renderers configuration
-----------------------
    Default renderer: 'kaggle'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

		– MATPLOTLIB VERSION: 3.7.2


Custom TB Handler failed, unregistering


In [12]:
# Create a list with the filepaths for training and testing
train_dir = Path('/kaggle/input/fruit-and-vegetable-image-recognition/train')
train_filepaths = list(train_dir.glob(r'**/*.jpg'))

test_dir = Path('/kaggle/input/fruit-and-vegetable-image-recognition/test')
test_filepaths = list(test_dir.glob(r'**/*.jpg'))

val_dir = Path('/kaggle/input/fruit-and-vegetable-image-recognition/validation')
val_filepaths = list(test_dir.glob(r'**/*.jpg'))

def proc_img(filepath):
    """ Create a DataFrame with the filepath and the labels of the pictures
    """

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # Concatenate filepaths and labels
    df = pd.concat([filepath, labels], axis=1)

    # Shuffle the DataFrame and reset index
    df = df.sample(frac=1).reset_index(drop = True)
    
    return df

train_df = proc_img(train_filepaths)
test_df = proc_img(test_filepaths)
val_df = proc_img(val_filepaths)

In [13]:
print('-- Training set --\n')
print(f'Jumlah Foto : {train_df.shape[0]}\n')
print(f'Jumlah Label : {len(train_df.Label.unique())}\n')
print(f'Label: {train_df.Label.unique()}')

-- Training set --

Jumlah Foto : 2780

Jumlah Label : 36

Label: ['pomegranate' 'pineapple' 'kiwi' 'peas' 'lettuce' 'carrot' 'mango'
 'paprika' 'raddish' 'apple' 'garlic' 'cauliflower' 'banana' 'beetroot'
 'chilli pepper' 'spinach' 'watermelon' 'potato' 'turnip' 'ginger'
 'sweetcorn' 'lemon' 'eggplant' 'sweetpotato' 'capsicum' 'cucumber'
 'jalepeno' 'corn' 'onion' 'soy beans' 'bell pepper' 'orange' 'cabbage'
 'tomato' 'pear' 'grapes']


In [ ]:
train_df.head()

In [ ]:
# Display some pictures of the dataset
fig, axes = plt.subplots(nrows=6, ncols=6, figsize=(8, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_df.Filepath[i]))
    ax.set_title(train_df.Label[i], fontsize = 12)
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
X = []
Y = []

In [ ]:
def transfer_between_folders(source1, source2, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")
        
def transfer_all_class_between_folders(source1, source2, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    for label in config.LABEL:
        transfer_between_folders(os.path.join(source1, config.LABEL),
                                 os.path.join(
                                      source2, config.LABEL),
                                 split_rate)

In [ ]:
class config:
    # specify the paths to datasets
    DATA_DIR = Path('/kaggle/input/fruit-and-vegetable-image-recognition')
    ROOT_DIR = Path('/kaggle/working/')
    TRAIN_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/train"
    TEST_DIR = "/kaggle/input/fruit-and-vegetable-image-recognition/test"
    VAL_DIR = '/kaggle/input/fruit-and-vegetable-image-recognition/validation'

    # set the input height and width
    INPUT_HEIGHT = 256
    INPUT_WIDTH = 256

    # set the input heig/ht and width
    IMAGENET_MEAN = [0.485, 0.456, 0.406]
    IMAGENET_STD = [0.229, 0.224, 0.225]
    
    BATCH_SIZE = 128
    MODEL_NAME = 'regnetY800'
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    TRAINING_PARAMS = 'training_hyperparams/default_train_params'
    
    NUM_CLASSES = len(train_df.Label.unique())
    LABEL = train_df.Label.unique()

    CHECKPOINT_DIR = 'checkpoints'
    

In [ ]:
# initialize our data augmentation functions
resize = transforms.Resize(size=(config.INPUT_HEIGHT,config.INPUT_WIDTH))
make_tensor = transforms.ToTensor()
normalize = transforms.Normalize(mean=config.IMAGENET_MEAN, std=config.IMAGENET_STD)
center_cropper = transforms.CenterCrop((config.INPUT_HEIGHT,config.INPUT_WIDTH))
random_horizontal_flip = transforms.RandomHorizontalFlip(p=0.75)
random_vertical_flip = transforms.RandomVerticalFlip(p=0.75)
random_rotation = transforms.RandomRotation(degrees=90)
random_crop = transforms.RandomCrop(size=(200,200))
augmix = transforms.AugMix(severity = 3, mixture_width=3, alpha=0.2)
auto_augment = transforms.AutoAugment()
random_augment = transforms.RandAugment()

# initialize our training and validation set data augmentation pipeline
train_transforms = transforms.Compose([
  resize, 
  auto_augment,
  augmix,
  random_augment,
  make_tensor,
  normalize
])

val_transforms = transforms.Compose([resize, make_tensor, normalize])

In [ ]:
def create_dataloaders(
    train_dir: str, 
    val_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    val_transform:  transforms.Compose,
    test_transform:  transforms.Compose,
    batch_size: int, 
    num_workers: int=2
):
  
  # Use ImageFolder to create dataset
  train_data = datasets.ImageFolder(train_dir, transform=train_transform)
  val_data = datasets.ImageFolder(val_dir, transform=val_transform)
  test_data = datasets.ImageFolder(test_dir, transform=val_transform)  

  print(f"[INFO] training dataset contains {len(train_data)} samples...")
  print(f"[INFO] validation dataset contains {len(val_data)} samples...")
  print(f"[INFO] test dataset contains {len(test_data)} samples...")

  # Get class names
  class_names = train_data.classes
  print(f"[INFO] dataset contains {len(class_names)} labels...")

  # Turn images into data loaders
  print("[INFO] creating training and validation set dataloaders...")
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      drop_last=True,
      num_workers=num_workers,
      pin_memory=True,
      persistent_workers=True
  )
  val_dataloader = DataLoader(
      val_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
      drop_last=False,
      persistent_workers=True
  )

  return train_dataloader, val_dataloader, test_dataloader, class_names

In [ ]:
train_dataloader, valid_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=config.TRAIN_DIR,
                                                                     val_dir=config.VAL_DIR,
                                                                     test_dir=config.TEST_DIR,
                                                                     train_transform=train_transforms,
                                                                     val_transform=val_transforms,
                                                                     test_transform=val_transforms,
                                                                     batch_size=config.BATCH_SIZE)

NUM_CLASSES = len(class_names)

In [ ]:
training_params =  training_hyperparams.get(config.TRAINING_PARAMS)

In [ ]:


training_params = {
             # Your training params
    "sg_logger": "wandb_sg_logger", # Weights&Biases Logger, see class super_gradients.common.sg_loggers.wandb_sg_logger.WandBSGLogger for details
    "sg_logger_params":             # Params that will be passes to __init__ of the logger super_gradients.common.sg_loggers.wandb_sg_logger.WandBSGLogger
      {
        "project_name": "Fruit classification with regnetY800", # W&B project name
        "save_checkpoints_remote": True,
        "save_tensorboard_remote": True,
        "save_logs_remote": True,
        "entity": "rpl"   # Optional: In case your experiment tracking is not hosted at wandb servers
      }
    
}
# To reduce clutter in the notebook I've turned the verbosity off, you can turn it on to see the full output
early_stop_acc = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="Accuracy", mode="max", patience=7, verbose=True)
early_stop_val_loss = EarlyStop(Phase.VALIDATION_EPOCH_END, monitor="LabelSmoothingCrossEntropyLoss", mode="min", patience=7, verbose=True)

training_params["train_metrics_list"] = [Accuracy(), Top5()]
training_params["valid_metrics_list"] = [Accuracy(), Top5()]


# Set the silent mode to True to reduce clutter in the notebook, you can turn it on to see the full output
training_params["silent_mode"] = False
training_params["optimizer"] = 'AdamW'
training_params["criterion_params"] = {'smooth_eps': 0.20}
training_params["max_epochs"] = 250
training_params["initial_lr"] = 0.0001
training_params["loss"] = "cross_entropy"

In [ ]:
print(training_params)

In [ ]:
model = models.get(config.MODEL_NAME, num_classes = config.NUM_CLASSES, pretrained_weights='imagenet')

In [ ]:
full_model_trainer = Trainer(experiment_name='Fruit classification with regnetY800', ckpt_root_dir=config.CHECKPOINT_DIR)

In [ ]:
full_model_trainer.train(model=model, 
              training_params=training_params, 
              train_loader=train_dataloader,
              valid_loader=valid_dataloader)

In [ ]:
best_full_model = models.get(config.MODEL_NAME,
                        num_classes=config.NUM_CLASSES,
                        checkpoint_path=os.path.join(full_model_trainer.checkpoints_dir_path, "ckpt_best.pth"))

In [ ]:
def pred_and_plot_image(image_path: str, 
                        subplot: Tuple[int, int, int],  # subplot tuple for `subplot()` function
                        class_names: List[str] = class_names,
                        model: torch.nn.Module = best_full_model,
                        image_size: Tuple[int, int] = (config.INPUT_HEIGHT, config.INPUT_WIDTH),
                        transform: torchvision.transforms = None,
                        device: torch.device=config.DEVICE):

    if isinstance(image_path, pathlib.PosixPath):
      img = Image.open(image_path)
    else: 
      img = Image.open(requests.get(image_path, stream=True).raw)

    # create transformation for image (if one doesn't exist)
    if transform is None:
        transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=config.IMAGENET_MEAN,
                                 std=config.IMAGENET_STD),
        ])
    transformed_image = transform(img)

    # make sure the model is on the target device
    model.to(device)

    # turn on model evaluation mode and inference mode
    model.eval()
    with torch.inference_mode():
        # add an extra dimension to image (model requires samples in [batch_size, color_channels, height, width])
        transformed_image = transformed_image.unsqueeze(dim=0)

        # make a prediction on image with an extra dimension and send it to the target device
        target_image_pred = model(transformed_image.to(device))

    # convert logits -> prediction probabilities (using torch.softmax() for multi-class classification)
    target_image_pred_probs = torch.softmax(target_image_pred, dim=1)

    # convert prediction probabilities -> prediction labels
    target_image_pred_label = torch.argmax(target_image_pred_probs, dim=1)

    # actual label
    ground_truth = PurePath(image_path).parent.name

    # plot image with predicted label and probability 
    plt.subplot(*subplot)
    plt.imshow(img)
    if isinstance(image_path, pathlib.PosixPath):
        title = f"Ground Truth: {ground_truth} | Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    else:
        title = f"Pred: {class_names[target_image_pred_label]} | Prob: {target_image_pred_probs.max():.3f}"
    plt.title("\n".join(textwrap.wrap(title, width=20)))  # wrap text using textwrap.wrap() function
    plt.axis(False)
    


In [ ]:
image = 'https://th.bing.com/th/id/R.ef9ff3be84f85e3ca222b8d9b16198b5?rik=DbNJVlgcB%2fV1LA&riu=http%3a%2f%2fcottageintheoaks.files.wordpress.com%2f2011%2f05%2fcorn-featured-vegetable.jpg&ehk=ZCXWFaYmBoNkuXISD7SebYWc1krNDd5ZDI4%2fGiA56Ug%3d&risl=&pid=ImgRaw&r=0'
pred_and_plot_image(image_path= image, subplot=(1, 1, 1))

In [ ]:
image = 'https://www.gardeningknowhow.com/wp-content/uploads/2021/05/organic-pears-on-a-tree.jpg'
pred_and_plot_image(image_path= image, subplot=(1, 1, 1))

In [ ]:
image = 'https://th.bing.com/th/id/R.f84b480a518fc98e28c3df89b3861954?rik=px288gPNXFvnJw&riu=http%3a%2f%2fstatic.republika.co.id%2fuploads%2fimages%2finpicture_slide%2fjahe-_140311172716-448.jpg&ehk=HQle2NHXKrfgV67BHSrZRbGQmsDU61RxBkOEkzaXt28%3d&risl=&pid=ImgRaw&r=0'
pred_and_plot_image(image_path= image, subplot=(1, 1, 1))

In [ ]:
image = 'https://th.bing.com/th/id/OIP.hHu4F3X3vzArSEHQskL02AHaFj?w=249&h=187&c=7&r=0&o=5&dpr=1.3&pid=1.7'
pred_and_plot_image(image_path= image, subplot=(1, 1, 1))

In [ ]:
image = 'https://i.pinimg.com/originals/48/15/28/4815281f9b9351265fbe2b0173d5aa4b.jpg'
pred_and_plot_image(image_path= image, subplot=(1, 1, 1))

In [ ]:
from sklearn.metrics import confusion_matrix

# Set model to evaluation mode
best_full_model.eval()

# Create empty lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop over batches in test dataloader, make predictions, and append true and predicted labels to lists
for images, labels in test_dataloader:
    images = images.to(config.DEVICE)
    labels = labels.to(config.DEVICE)
    with torch.no_grad():
        outputs = best_full_model(images)
        _, predicted = torch.max(outputs.data, 1)
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

# Calculate confusion matrix, precision, and recall
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Create figure and axis objects with larger size and font size
fig, ax = plt.subplots(figsize=(12, 10))
plt.rcParams.update({'font.size': 16})

# Create heatmap of confusion matrix
im = ax.imshow(conf_matrix, cmap='Blues')

# Add colorbar to heatmap
cbar = ax.figure.colorbar(im, ax=ax)

# Set tick labels and axis labels with larger font size
ax.set_xticks(np.arange(len(class_names)))
ax.set_yticks(np.arange(len(class_names)))
ax.set_xticklabels(class_names, fontsize=14)
ax.set_yticklabels(class_names, fontsize=14)
ax.set_xlabel('Predicted label', fontsize=16)
ax.set_ylabel('True label', fontsize=16)

# Rotate tick labels and set alignment
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add text annotations to heatmap
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if conf_matrix[i, j] >= -1:  # Modify threshold value as needed
            text = ax.text(j, i, conf_matrix[i, j],
                           ha="center", va="center", color="y", fontsize=16)
        else:
            text = ax.text(j, i, "",
                           ha="center", va="center", color="y")

# Add title to plot with larger font size
ax.set_title("Confusion matrix", fontsize=20)

# Show plot
plt.show()

In [ ]:
model.prep_model_for_conversion(input_size=[1, 3, config.INPUT_HEIGHT, config.INPUT_WIDTH])
# Convert model to onnx
torch.onnx.export(model, dummy_input,  "yolo_nas_m.onnx")

@misc{supergradients,
  doi = {10.5281/ZENODO.7789328},
  url = {https://zenodo.org/record/7789328},
  author = {Aharon,  Shay and {Louis-Dupont} and {Ofri Masad} and Yurkova,  Kate and {Lotem Fridman} and {Lkdci} and Khvedchenya,  Eugene and Rubin,  Ran and Bagrov,  Natan and Tymchenko,  Borys and Keren,  Tomer and Zhilko,  Alexander and {Eran-Deci}},
  title = {Super-Gradients},
  publisher = {GitHub},
  journal = {GitHub repository},
  year = {2021},
}